In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from PIL import Image
from collections import Counter

In [2]:
dataset_path = "E:/Codes/Garbage Classification/dataset-resized"
classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=classes,
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=classes,
    subset='validation'
)

print("Class indices:", train_generator.class_indices)

Found 0 images belonging to 6 classes.
Found 0 images belonging to 6 classes.
Class indices: {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


In [4]:
class_counts = Counter()

for root, dirs, files in os.walk(dataset_path):
    if os.path.basename(root) in ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']:
        class_counts[os.path.basename(root)] += len(files)

print("Class distribution:", dict(class_counts))
total = sum(class_counts.values())
class_weights = {
    0: total / (6 * class_counts['cardboard']),
    1: total / (6 * class_counts['glass']),
    2: total / (6 * class_counts['metal']),
    3: total / (6 * class_counts['paper']),
    4: total / (6 * class_counts['plastic']),
    5: total / (6 * class_counts['trash'])
}

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Class distribution: {'trash': 137, 'cardboard': 403, 'glass': 501, 'metal': 410, 'paper': 594, 'plastic': 482}
Found 2022 images belonging to 2 classes.
Found 505 images belonging to 2 classes.


In [5]:
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')
])

base_model.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
class_counts = {
    'cardboard': 403,
    'glass': 501,
    'metal': 410,
    'paper': 594,
    'plastic': 482,
    'trash': 137
}

total = sum(class_counts.values())
class_weights = {
    list(train_generator.class_indices.keys()).index(cls): total / (len(class_counts) * count)
    for cls, count in class_counts.items()
}

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3)
    ]
)

ValueError: 'cardboard' is not in list

In [ ]:
test_loss, test_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {test_acc:.4f}")
print(f"Validation Loss: {test_loss:.4f}")

In [ ]:
class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

def predict_garbage(image_path, confidence_threshold=0.7):
    try:
        img = Image.open(image_path).convert('RGB').resize((224, 224))
        img_array = tf.keras.applications.mobilenetv2.preprocess_input(
            np.expand_dims(np.array(img), axis=0))
        
        pred = model.predict(img_array)[0]
        pred_idx = np.argmax(pred)
        confidence = np.max(pred)
        
        if confidence < confidence_threshold:
            top3 = np.argsort(pred)[-3:][::-1]
            return {
                'status': 'uncertain',
                'predictions': [(class_names[i], float(pred[i])) for i in top3]
            }
            
        return {
            'status': 'confident',
            'prediction': class_names[pred_idx],
            'confidence': float(confidence)
        }
        
    except Exception as e:
        return {'status': 'error', 'message': str(e)}

In [ ]:
result = predict_garbage("recyclable/plastic/plastic14.jpg")

if result['status'] == 'confident':
    print(f"Prediction: {result['prediction']} ({result['confidence']:.1%})")
elif result['status'] == 'uncertain':
    print("Top predictions:")
    for pred, conf in result['predictions']:
        print(f"- {pred}: {conf:.1%}")
else:
    print(f"Error: {result['message']}")

In [ ]:
model.save("garbage_classifier_v2.h5")
print("Model saved successfully")